# HW4 No Stop Words Version

Submitted by Russel Jade Tumanon

# Import Libraries

In [1]:
#Standard Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sklearn
import sklearn.metrics

#For Preprocessing and results
import email
import re
from collections import Counter
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix

# 1. Preprocessing

## Creating Table

In [2]:
#Creating table or dataframe to easily view the parts
labels_file = []
labels = []
path = []
temp = []
email_contents = []

#Setting DataFrame
emails = pd.DataFrame()

#Open labels file
with open('labels') as file:
    labels_file = file.readlines()

#Remove New lines   
for i in labels_file:
    temp.append(i.replace("\n", ""))
    
labels_file = temp

#Get the label, addr/path, and email contents
for i in labels_file:
    label, separator, addr = i.partition(" ")
    addr = addr[3:]
    path.append(addr)
    labels.append(label)

    #Put each email content to email_contents
    with open(addr, "r", errors='ignore') as currFile:
        contents = currFile.read()
        email_contents.append(contents)
        
        currFile.close()

#Assigning colum names 
emails["label"] = labels
emails["path"] = path
emails["contents"] = email_contents

#Print first 5 
emails.head(5)

,label,path,contents
0,ham,data/000/000,Received: from rodan.UU.NET by aramis.rutgers....
1,spam,data/000/001,Received: from unknown (HELO groucho.cs.psu.ed...
2,spam,data/000/002,Received:\n\tfrom 24-151-178-89.dhcp.kgpt.tn.c...
3,ham,data/000/003,Received: from psuvax1.cs.psu.edu ([130.203.2....
4,spam,data/000/004,Received: from 201-1-198-159.dsl.telesp.net.br...


## Cleaning the Dataset 

### Functions to Format Email Contents

In [3]:
#Creating a function for string emails
def email_messg(email_str):
    message_str = email.message_from_string(email_str)
    mssg = ''
    
    #Get Payload
    try:
        for payload in message_str.get_payload():
            mssg = payload.get_payload()
    except:
        mssg = message_str.get_payload()

    #Calls the multipart function if it's not a string
    if(type(mssg) != str):
        mssg = multipart(email_str)
    return str(mssg)

#Creating a function for multipart emails
def multipart(email_content_str):
    email_body = email.message_from_string(email_content_str)
    content = ""
    
    #Get the body for multipart emails
    if email_body.is_multipart():
        
        #Get the content type and disposition
        for i in email_body.walk():
            content_type = i.get_content_type()
            content_dispo = str(i.get('Content-Disposition'))
            
            #Will skip if the content type is text/plain and content disposition do not contain attachment
            if content_type == 'text/plain' and 'attachment' not in content_dispo:
                
                #Decode payload
                content = i.get_payload(decode=True)
                break
                
    #If not multipart
    else:
        
        #Decode payload
        content = email_body.get_payload(decode=True)
    try:
        
         #Decode content
        content = content.decode()
    except:
        try:
            
            #Decode content 'latin-1'
            content = content.decode('latin-1')
        except:
            pass
    return(content)

### Store all Formatted Email Contents to a List

In [4]:
#Store all contents to email_contents
email_contents = []
for i in emails.contents:
    email_contents.append(email_messg(i))

#Sample
print(f"Formatted Result Sample:")
email_contents[0]

Formatted Result Sample:


'The mailing list I queried about a few weeks ago is now up and\nrunning.  I have also set up an archive server; see below.  The\nfollowing is the official welcome to the list message at the moment.\n\nJoe Buehler\n\n--------------------\n\nThis mailing list is for people who desire serious, orthodox\ndiscussion of the Roman Catholic religion.  I assume that it will\ncater mainly to Catholics, but everyone else is welcome, provided they\noperate within the below guidelines.  My own interests have a\ndoctrinal bent, but I\'m certainly not going to limit this list to just\nthat sort of discussion.\n\nHaving participated in USENET religion groups for about 5 years now,\none of my primary observations about Catholics on the net is that they\ndo not know their religion very well.  My hope is that this list might\nhelp remedy this problem to some extent.  I would like to make this a\nnet resource available for Catholics who want to know more about their\nreligion.\n\nAs far as moderation pol

### NO Stop Words 

### Functions to  Clean the Email Contents

In [5]:
#Creating a function to clean email contents
#Stop Words not included
def clean(email_content_str):

    #Lower case all words
    email_content_str = email_content_str.lower().split()
    #Joining to string
    email_content_str = ' '.join(email_content_str)   
    
    #Removing non-alphanumeric characters
    email_content_str = re.sub(r'[^a-zA-Z0-9]', ' ', email_content_str)

    #Removing numbers
    email_content_str = re.sub('\d+', ' ', email_content_str)
    
    #Removing website links
    email_content_str = re.sub(r'\bhttp\S*\b', ' ', email_content_str)
    email_content_str = re.sub(r'\bwww\S*', ' ', email_content_str)
    
    #Return cleaned email
    return(email_content_str)

#Creating a function to clean evey email content
def clean_emails(email_contents):
    cleaned_emails = []
    for email_content_str in email_contents:
        cleaned_emails.append(clean(email_content_str))
    
    return cleaned_emails

cleaned_emails = clean_emails(email_contents)

#Sample
print(f"Cleaned Result Sample:")
cleaned_emails[0]

Cleaned Result Sample:


'the mailing list i queried about a few weeks ago is now up and running  i have also set up an archive server  see below  the following is the official welcome to the list message at the moment  joe buehler                      this mailing list is for people who desire serious  orthodox discussion of the roman catholic religion  i assume that it will cater mainly to catholics  but everyone else is welcome  provided they operate within the below guidelines  my own interests have a doctrinal bent  but i m certainly not going to limit this list to just that sort of discussion  having participated in usenet religion groups for about   years now  one of my primary observations about catholics on the net is that they do not know their religion very well  my hope is that this list might help remedy this problem to some extent  i would like to make this a net resource available for catholics who want to know more about their religion  as far as moderation policy goes  the catholic church is n

### Update DataFrame

In [6]:
#Update dataframe contents
emails["cleaned_contents"] = cleaned_emails
emails.head(5)

,label,path,contents,cleaned_contents
0,ham,data/000/000,Received: from rodan.UU.NET by aramis.rutgers....,the mailing list i queried about a few weeks a...
1,spam,data/000/001,Received: from unknown (HELO groucho.cs.psu.ed...,luxury watches buy your own rolex for only ...
2,spam,data/000/002,Received:\n\tfrom 24-151-178-89.dhcp.kgpt.tn.c...,academic qualifications available from prestig...
3,ham,data/000/003,Received: from psuvax1.cs.psu.edu ([130.203.2....,greetings all this is to verify your subscrip...
4,spam,data/000/004,Received: from 201-1-198-159.dsl.telesp.net.br...,html head meta equiv content language ...


## Splitting the Dataset

### Train Set for Ham and Spam

In [7]:
#For Train remember that it has 21,300 emails so we count index 0 to 21300
email_train = emails[0:21300]

#For ham training set 
email_train_ham = email_train[email_train['label'].str.contains('ham')]
#For spam training set 
email_train_spam = email_train[email_train['label'].str.contains('spam')]

### Test Set 

In [8]:
#For Test remember that it has 16,522 emails so we count from train's last index to the total emails + 1 = 37823
email_test = emails[21300:37823]

### Checking Emails of Train and Test Sets

In [9]:
#Check Train Set
print("Train set shape:", email_train.shape)
print("Trainset shape for ham:", email_train_ham.shape)
print("Train set shape for spam:", email_train_spam.shape)

#Checks if the number of emails from the training dataset of ham and spam is equal to the total train emails (21300)
if (email_train_ham.shape[0] + email_train_spam.shape[0] == 21300):
    print("\nSplitting Train dataset for ham and spam successful!")
else:
    print("Emails do not add up!")

#Check Test Set 
print("\nTest set shape:", email_test.shape)

Train set shape: (21300, 4)
Trainset shape for ham: (7523, 4)
Train set shape for spam: (13777, 4)

Splitting Train dataset for ham and spam successful!

Test set shape: (16522, 4)


## Occurrence of Unique words

In [10]:
#Counting the occurrence of each unique word in cleaned_contents using a Counter
unique = Counter(" ".join(email_train['cleaned_contents']).split()).most_common(10000)
dictionary = pd.DataFrame(unique, columns = ['word', 'occurence'])

dictionary.head(10000)

,word,occurence
0,d,242061
1,b,241090
2,e,228292
3,a,197327
4,div,157355
...,...,...
9995,shin,31
9996,flip,31
9997,marshall,31
9998,objectives,31


# 2. Creating the feature matrices

### For Ham Train Set

In [11]:
contents_list_ham = email_train_ham['cleaned_contents'].tolist()
dict_words = dictionary['word'].tolist()

#Create empty feature matrix
feature_matrix_ham = np.zeros((len(contents_list_ham), len(dict_words)))

#Loop over the emails
for i, email in enumerate(contents_list_ham):
    
    #Check if the word in the dictionary appears in the email
    for j, word in enumerate(dict_words):
        if word in email:
            #If it appears, set the value to 1
            feature_matrix_ham[i, j] = 1
    else:
        #Set to 0
        feature_matrix_ham[i, j] = 0

feature_matrix_ham

array([[1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       ...,
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.]])

### For Spam Train Set

In [12]:
contents_list_spam = email_train_spam['cleaned_contents'].tolist()

#Create empty feature matrix
feature_matrix_spam = np.zeros((len(contents_list_spam), len(dict_words)))

#Loop over the emails
for i, email in enumerate(contents_list_spam):
    
    #Check if the word in the dictionary appears in the email
    for j, word in enumerate(dict_words):
        if word in email:
        
            #If it appears, set the value to 1
            feature_matrix_spam[i, j] = 1
    else:
        
        #Set to 0
        feature_matrix_spam[i, j] = 0

feature_matrix_spam

array([[1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.]])

# 3. Computing the Priors

### Preparing Variabels

In [13]:
#The number of ham emails in the training set
train_size_ham = email_train_ham.shape[0]

#The number of spam emails in the training set
train_size_spam = email_train_spam.shape[0]

#The total number of emails in train set
total_email_size = email_train.shape[0]

### Solution for Prior Probabilities of Ham and Spam

In [14]:
#Solution for the prior probabilities for ham
prior_ham =  train_size_ham / total_email_size

#Solution for the prior probabilities for spam
prior_spam = train_size_spam / total_email_size

#Showing Results
print(f"P(c = ham) = {prior_ham}")
print(f"P(c = spam) = {prior_spam}")

P(c = ham) = 0.3531924882629108
P(c = spam) = 0.6468075117370892


# 4. Computing the Likelihood of each word

### Preparing Variabels

In [15]:
#Add all the spam words
words_train_spam_sum = np.sum(feature_matrix_spam, axis=0)

#Add all ham words
words_train_ham_sum  = np.sum(feature_matrix_ham, axis=0)

#Total sum of spam words
words_train_spam_sum_total = words_train_spam_sum.sum()

#Total sum of ham words
words_train_ham_sum_total = words_train_ham_sum.sum()

#For laplace smoothing
laplace_smoothing = 1 

#Create blank dicts
likelihood_ham = {}
likelihood_spam = {}
#formula based on the given formula in the instructions

### Solution for the Likelihood of Each Word

In [16]:
uniq_words = dictionary['word'].tolist()

for i in range(len(uniq_words)):

    #Getting each word's occurance (spam)
    occur_spam = (words_train_spam_sum[i] + laplace_smoothing) / (words_train_spam_sum_total + laplace_smoothing * len(uniq_words))
    likelihood_spam[uniq_words[i]] = occur_spam  
    
    #Getting each word's occurance (ham)
    occur_ham = (words_train_ham_sum[i] + laplace_smoothing) / (words_train_ham_sum_total + laplace_smoothing * len(uniq_words))
    likelihood_ham[uniq_words[i]] = occur_ham

#Sample
#likelihood_spam
first_key_spam, first_value_spam = next(iter(likelihood_spam.items()))
print(f"Result Sample\n\nLikelyhood of \"{first_key_spam}\" occuring as spam is {first_value_spam}.")

#likelihood_ham
first_key_ham, first_value_ham = next(iter(likelihood_ham.items()))
print(f"\nLikelyhood of \"{first_key_ham}\" occuring as ham is {first_value_ham}.")

Result Sample

Likelyhood of "d" occuring as spam is 0.002734251142013482.

Likelyhood of "d" occuring as ham is 0.0026568330750450513.


# 5. Classifying the emails

In [17]:
#Functiom to classify if it's a spam or ham based on the probability of its word's occurrance
contents_list = emails['cleaned_contents'].tolist()
def email_classify(prior_spam, prior_ham, contents_list, likelihood_spam, likelihood_ham, uniq_words):
    
    #Converting the prior probabilities of spam and ham to log probabilities for numerical stability
    log_prob_spam = np.log(prior_spam)
    log_prob_ham = np.log(prior_ham)
    
    #Split the cleaned email contents into a list of individual words
    body = str(contents_list).split()
    
    for word in body:

        #Update the log probabilities if the word is in the list of unique words
        if word in uniq_words:
            log_prob_spam += np.log(likelihood_spam[word])            
            log_prob_ham += np.log(likelihood_ham[word])

    #Check if it is more likely to be spam or ham
    if log_prob_spam > log_prob_ham:
        return "spam"
    else:
        return "ham"

# 6. Testing the Classifier

### Testing if the predicted label and actual label is correct

In [18]:
#Test it to all the emails
predicted_label = []

#Go through each email content and classify them
for content in contents_list:
    predicted = email_classify(prior_spam, prior_ham, content, likelihood_spam, likelihood_ham, uniq_words)
    predicted_label.append(predicted)

### Add to the DataFrame 

In [19]:
#Insert to the right side of label to easily make a comparison
emails.insert(1, 'predicted label', predicted_label)
emails.head(1000)

,label,predicted label,path,contents,cleaned_contents
0,ham,ham,data/000/000,Received: from rodan.UU.NET by aramis.rutgers....,the mailing list i queried about a few weeks a...
1,spam,spam,data/000/001,Received: from unknown (HELO groucho.cs.psu.ed...,luxury watches buy your own rolex for only ...
2,spam,spam,data/000/002,Received:\n\tfrom 24-151-178-89.dhcp.kgpt.tn.c...,academic qualifications available from prestig...
3,ham,ham,data/000/003,Received: from psuvax1.cs.psu.edu ([130.203.2....,greetings all this is to verify your subscrip...
4,spam,spam,data/000/004,Received: from 201-1-198-159.dsl.telesp.net.br...,html head meta equiv content language ...
...,...,...,...,...,...
995,spam,spam,data/003/095,Received: from mail.oh-oku.com (61-30-232-94.s...,b ...
996,ham,ham,data/003/096,Received: from unicorn.acs.ttu.edu (unicorn.ac...,wind erosion list members the following annou...
997,ham,ham,data/003/097,Received: from unicorn.acs.ttu.edu (unicorn.ac...,we want to update some information on land dam...
998,spam,spam,data/003/098,Received: from mail.oh-oku.com (61-30-232-94.s...,b ...


### Count how many correct emails were classified

In [20]:
#Count rows that are equal
count = (emails['label'].eq(emails['predicted label'])).sum()
total = len(contents_list)

#Convert to Percentage
percentage = count/total * 100
percentage = round(percentage, 2)

print(f"Correctly classified emails account to {count}/{total} or {percentage}%")

Correctly classified emails account to 35261/37822 or 93.23%


# 7. Performance Evaluation Without Stop Words

In [21]:
#Performance evaluation accuracy, recall, and precision
label_actual = emails['label'].to_numpy()
label_pred = emails['predicted label'].to_numpy()

#For Accuracy
#Round to 2 decimal places and convert to percentage
accuracy_result = accuracy_score(label_actual, label_pred)
accuracy_result_p = round(accuracy_result * 100, 2)

#For Recall
recall_result = recall_score(label_actual, label_pred, pos_label="ham")
recall_result_p = round(recall_result * 100, 2)

#For Precision
precision_result = precision_score(label_actual, label_pred, pos_label="ham")
precision_result_p = round(precision_result *100, 2)

#Results
print("Performance Evaluation:\n")
print(f"The Accuracy Score is {accuracy_result} or {accuracy_result_p}%")
print(f"The Recall Score is {recall_result} or {recall_result_p}%")
print(f"The Precision Score is {precision_result} or {precision_result_p}%")

Performance Evaluation:

The Accuracy Score is 0.9322880862989794 or 93.23%
The Recall Score is 0.9181254841208366 or 91.81%
The Precision Score is 0.8873998652392004 or 88.74%


# Confusion Matrix

In [22]:
#Confusion matrix
confusion_mat = confusion_matrix(label_actual,label_pred)
print("The Confusion Matrix:\n")
print(confusion_mat)

The Confusion Matrix:

[[11853  1057]
 [ 1504 23408]]


# =======================================

### Showing Table Evaluation Result with Stop Words and Without

In [23]:
#To create a table, we create a dictionary with the data for the DataFrame
acc = str(accuracy_result_p) + "%"
rec = str(recall_result_p) + "%"
prec = str(precision_result_p) + "%"

data = {
    "With Stop Words": ["94.26%", "91.12%", "91.99%"],
    "No Stop Words": [acc, rec, prec]
}
table = pd.DataFrame(data, index=["Accuracy", "Recall", "Precision"])

table.head()

,With Stop Words,No Stop Words
Accuracy,94.26%,93.23%
Recall,91.12%,91.81%
Precision,91.99%,88.74%


The Table shows no difference whether we add the stop words or not in our results.

# =======================================

# End